In [1]:
# !huggingface-cli login
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!pip install transformers
!pip install SentencePiece
!pip install openpyxl
!pip install xlsxwriter

In [4]:
! git clone https://github.com/csebuetnlp/xl-sum
! pip install --upgrade -r ./xl-sum/seq2seq/requirements.txt
! pip install --upgrade transformers/
# install rouge module and dependecies
! pip install -r ./xl-sum/multilingual_rouge_scoring/requirements.txt
! python -m unidic download # for japanese segmentation
! pip install --upgrade ./xl-sum/multilingual_rouge_scoring
! python -m nltk.downloader punkt

Cloning into 'xl-sum'...
remote: Enumerating objects: 1156, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 1156 (delta 19), reused 30 (delta 12), pack-reused 1109
Receiving objects: 100% (1156/1156), 5.53 MiB | 12.05 MiB/s, done.
Resolving deltas: 100% (309/309), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 12.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.4/408.4 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.2 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=a0a3e9237a8a3a9173bcdf2b84135ac81f94383b03e

In [6]:
import datasets
import nltk
import numpy as np
import logging

logger = logging.getLogger(__name__)

In [7]:
CKPT = 'jannatul17/squad-bn-qgen-banglat5'
from transformers import AutoTokenizer, T5ForConditionalGeneration
from datasets import load_dataset
tokenizer_test = AutoTokenizer.from_pretrained(CKPT, use_auth_token=False)
model_test = T5ForConditionalGeneration.from_pretrained(CKPT, use_auth_token=False)

Downloading:   0%|          | 0.00/2.49k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.62M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/945M [00:00<?, ?B/s]

# Getting Dataset

In [8]:
from datasets import concatenate_datasets, load_dataset

train_data = load_dataset("csebuetnlp/squad_bn", split="train")
valid_data = load_dataset("csebuetnlp/squad_bn", split="validation")
test_data = load_dataset("csebuetnlp/squad_bn", split="test")

concat_dataset = concatenate_datasets([train_data, valid_data, test_data])
dataset=concat_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset squad_bn downloaded and prepared to /root/.cache/huggingface/datasets/csebuetnlp___squad_bn/squad_bn/0.0.1/17a6d6abc976f299afda17ca9b5ce08a022ecafabe24b3362e16a3093c32df4b. Subsequent calls will reuse this data.


In [9]:
del concat_dataset, train_data, valid_data, test_data

In [10]:

small_dataset= dataset.filter(lambda example: len(example['answers']['text'])>0)
small_dataset= small_dataset.filter(lambda example: len(example['context'])>300)

  0%|          | 0/124 [00:00<?, ?ba/s]

  0%|          | 0/72 [00:00<?, ?ba/s]

In [11]:
split_data = small_dataset.train_test_split(test_size=0.2, seed= None, shuffle=False)

In [12]:
dataset = split_data

In [13]:
del split_data

In [14]:
def format_dataset(example):
  
        return {'input': 'answer: ' + example['answers']['text'][0] + ' context: ' + example['context'], 'target': example['question']}

In [6]:
#test_data = load_dataset('csebuetnlp/squad_bn', split='test')

# small_dataset= test_data.filter(lambda example: len(example['answers']['text'])>0)
# small_dataset= small_dataset.filter(lambda example: len(example['context'])>300)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset squad_bn downloaded and prepared to /root/.cache/huggingface/datasets/csebuetnlp___squad_bn/squad_bn/0.0.1/17a6d6abc976f299afda17ca9b5ce08a022ecafabe24b3362e16a3093c32df4b. Subsequent calls will reuse this data.


In [7]:
def format_dataset(example):
    if (len(example['answers']['text'])>0):
        return {'input': 'answer: ' + example['answers']['text'][0] + ' context: ' + example['context'], 'target': example['question']}

In [15]:
dataset = dataset.map(format_dataset, remove_columns=dataset['train'].column_names)


  0%|          | 0/54150 [00:00<?, ?ex/s]

  0%|          | 0/13538 [00:00<?, ?ex/s]

In [19]:
test_data = dataset['test']

In [16]:
#dataset = test_data.map(format_dataset, remove_columns=test_data.column_names)

# # dataset = test_data.map(format_dataset)
#test_data['answers'][6]['text']
#dataset[0]
#small_dataset= dataset.filter(lambda example: len(example['answers']['text'])>0)
# small_dataset= small_dataset.filter(lambda example: len(example['context'])>300)


In [20]:
len(test_data)

13538

In [22]:
test_data[0]

{'input': 'answer: ১৯৮৭ context: ফলস্বরূপ, ১৯৭৯ সালে, সনি এবং ফিলিপস একটি নতুন ডিজিটাল অডিও ডিস্ক ডিজাইন করার জন্য প্রকৌশলীদের একটি যৌথ টাস্ক ফোর্স গঠন করে। ইঞ্জিনিয়ার কিস শুহামার ইমমিনক এবং তোশিতাদা দোই এর নেতৃত্বে, গবেষণাটি লেজার এবং অপটিক্যাল ডিস্ক প্রযুক্তিকে এগিয়ে নিয়ে যায়। এক বছর পরীক্ষা-নিরীক্ষা ও আলোচনার পর টাস্ক ফোর্স রেড বুক সিডি-ডিএ স্ট্যান্ডার্ড তৈরি করে। প্রথম প্রকাশিত হয় ১৯৮০ সালে। আইইসি কর্তৃক ১৯৮৭ সালে আন্তর্জাতিক মান হিসেবে আনুষ্ঠানিকভাবে এই মান গৃহীত হয় এবং ১৯৯৬ সালে বিভিন্ন সংশোধনী মানের অংশ হয়ে ওঠে।',
 'target': 'কখন আইইসি রেড বুক সিডি-ডিএকে একটি আন্তর্জাতিক মান হিসাবে চালু করেছিল?'}

In [17]:
# import torch
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model_test.to(device)
# # To manage one sentence
# prompt = "answer: Pseudozizeeria maha context: ধুপি (বৈজ্ঞানিক নাম: Pseudozizeeria maha(Kollar))[1] এক প্রজাতির ছোট আকারের প্রজাপতি যার শরীর ও ডানা হালকা ধূসর-খয়েরি বর্ণের এবং বিন্দু দেখা যায়। এরা 'লাইসিনিডি' গোত্রের এবং 'পলিয়োম্যাটিনি' উপগোত্রের সদস্য।"
# input_ids = tokenizer_test(prompt, return_tensors="pt").input_ids.to(device)
# generated_ids = model_test.generate(input_ids,
#                                     num_beams=7,
#                                     num_return_sequences=7,
# #                                     top_k=10,
# #                                     top_p=.2,
# #                                     temperature=0.95,
                                    
#                                     max_length=200)
# for generated in generated_ids:
#     generated_text = tokenizer_test.decode(generated,skip_special_tokens=True)
# print(generated_text)

থুপির বৈজ্ঞানিক নাম কি?


In [23]:
# Function for handling multiple sentences
def custom_gen(model, tokenizer, input_text, max_length=50, do_sample=False, temperature=None, num_beams=None, top_k=None, top_p=None, early_stopping=True, num_return_sequences=1):
               input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
               generated_ids = model.generate(input_ids, max_length=max_length, do_sample=do_sample, top_k=top_k, temperature=temperature, num_beams=num_beams)
#generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
               return generated_ids

In [25]:
input_data = []
reference_question = []
answer = []
context = []
#for i in range(0, len(small_dataset)):
for i in range(0,5):
    input_data.append('answer: ' + small_dataset[i]['answers']['text'][0]  + ' context: ' + small_dataset[i]['context'])
    reference_question.append(small_dataset[i]['question'])
    answer.append(small_dataset[i]['answers']['text'][0])
    context.append(small_dataset[i]['context'])

In [27]:
reference_question

['কোন দেশে নরম্যান্ডি অবস্থিত?',
 'নরমান্ডিতে নরম্যানরা কখন ছিল?',
 'কোন দেশ থেকে নর্সের উৎপত্তি হয়েছিল?',
 'নর্স নেতা কে ছিলেন?',
 'নরম্যানরা প্রথম কোন শতাব্দীতে তাদের আলাদা পরিচয় লাভ করেছিল?']

In [43]:
# ('answer: ' + small_dataset[0]['answers']['text'][0]  + ' context: ' + small_dataset[0]['context'])
# input_data

In [12]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_test.to(device)

# Using the custom_gen. Will have to change for our code.
bn_pred_ques = {}
all_generated_ids = []
for prompt in input_data:
#     print(prompt)
    generated_ids = custom_gen(model_test,tokenizer_test,prompt,max_length=50,do_sample=True,top_k=40, top_p=0.95,num_return_sequences=20)
    all_generated_ids.append(generated_ids)
pred = []
pred_ques =[]
for text in all_generated_ids:
    for generated in text:
        generated_text = tokenizer_test.decode(generated,skip_special_tokens=True)
#         print(generated_text)
        pred.append(generated_text)
        bn_pred_ques[prompt] = pred
#     pred_ques.append(pred)
# print(pred)
# for generated in generated_ids:
#     generated_text = tokenizer_test.decode(generated,skip_special_tokens=True)
#     print(generated_text)
#     pred.append(generated_text)
#     bn_pred_ques[prompt] = pred



Token indices sequence length is longer than the specified maximum sequence length for this model (537 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
print(pred)

In [ ]:
print(reference_question)

In [13]:
from rouge_score import rouge_scorer, scoring

def add_newline_to_end_of_each_sentence(x):
        return "\n".join(nltk.sent_tokenize(x))



def extract_rouge_mid_statistics(dct):
        new_dict = {}
        for k1, v1 in dct.items():
            mid = v1.mid
            new_dict[k1] = {stat: round(getattr(mid, stat), 4) for stat in ["precision", "recall", "fmeasure"]}
        return new_dict


def calculate_rouge(
        pred_lns,
        tgt_lns,
        use_stemmer=True,
        rouge_keys=["rouge1", "rouge2", "rougeL", "rougeLsum"],
        return_precision_and_recall=False,
        bootstrap_aggregation=True,
        newline_sep=True,
       rouge_lang='bengali'
    ):
        
        logger.info("Rouge lang: " + str(rouge_lang))
        scorer = rouge_scorer.RougeScorer(
            rouge_keys, lang=rouge_lang,
            use_stemmer=use_stemmer
        )
        aggregator = scoring.BootstrapAggregator()
        for pred, tgt in zip(tgt_lns, pred_lns):
            # rougeLsum expects "\n" separated sentences within a summary
            if newline_sep:
                pred = add_newline_to_end_of_each_sentence(pred)
                tgt = add_newline_to_end_of_each_sentence(tgt)
            scores = scorer.score(pred, tgt)
            aggregator.add_scores(scores)

        if bootstrap_aggregation:
            result = aggregator.aggregate()
            if return_precision_and_recall:
                return extract_rouge_mid_statistics(result)  # here we return dict
            else:
                results_precision = {k: round(v.mid.precision * 100, 4) for k, v in result.items()}
                results_recall = {k: round(v.mid.recall * 100, 4) for k, v in result.items()}
                results_fmeasure = {k: round(v.mid.fmeasure * 100, 4) for k, v in result.items()}
            
            return {
                    "rouge1_precision": results_precision.get('rouge1'),
                    "rouge1_recall": results_recall.get('rouge1'),
                    "rouge1_fmeasure": results_fmeasure.get('rouge1'),

                    "rouge2_precision": results_precision.get('rouge2'),
                    "rouge2_recall": results_recall.get('rouge2'),
                    "rouge2_fmeasure": results_fmeasure.get('rouge2'),

                    "rougeL_precision": results_precision.get('rougeL'),
                    "rougeL_recall": results_recall.get('rougeL'),
                    "rougeL_fmeasure": results_fmeasure.get('rougeL'),
                
                    "rougeLsum_precision": results_precision.get('rougeLsum'),
                    "rougeLsum_recall": results_recall.get('rougeLsum'),
                    "rougeLsum_fmeasure": results_fmeasure.get('rougeLsum'),
                }
#                 return {k: round(v.mid.fmeasure * 100, 4) for k, v in result.items()}

        else:
            return aggregator._scores  # here we return defaultdict(list)

        
metric_fn = calculate_rouge
import numpy as np
from datasets import load_metric
metric = load_metric("sacrebleu")
meteor = load_metric('meteor')

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [14]:
def compute_metrics(ref, pred):
#     preds, labels = eval_preds
#     if isinstance(preds, tuple):
#         preds = preds[0]
        
#     # Replace -100 in the labels as we can't decode them.
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    
#     decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    
#     decoded_preds_rouge = tokenizer.batch_decode(preds, skip_special_tokens=True)
#     decoded_labels_rouge = tokenizer.batch_decode(labels, skip_special_tokens=True)

#    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(pred, ref)
    
    result_rouge = metric_fn(ref, pred)
    
    result_bleu = metric.compute(predictions=decoded_preds, references=decoded_labels)
    meteor_result = meteor.compute(predictions=decoded_preds, references=decoded_labels)
#     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    
    result = result_rouge
    result["sacrebleu"] = round(result_bleu['score'], 4)
    
    result["meteor"] = round(meteor_result["meteor"],4)
    
#     result["gen_len"] = np.mean(prediction_lens)
#     result = {k: round(v,4) for k, v in result.items()}
    return result

In [15]:
result = compute_metrics(reference_question, pred)

In [100]:
import pandas as pd

## convert your array into a dataframe
# df = pd.DataFrame (pred)
df = pd.DataFrame({'Reference_question':reference_question, 'Predicted_question':pred, 'Answer': answer, 'Context': context})
df_result = pd.DataFrame({'Rouge1_Precision': [result['rouge1_precision']], 'Rouge1_Recall': [result['rouge1_recall']],
                         'Rouge1_fmeasure': [result['rouge1_fmeasure']],
                         'Rouge2_Precision': [result['rouge2_precision']], 'Rouge2_Recall': [result['rouge2_recall']],
                         'Rouge2_fmeasure': [result['rouge2_fmeasure']],
                         'RougeL_Precision': [result['rougeL_precision']], 'RougeL_Recall': [result['rougeL_recall']],
                         'RougeL_fmeasure': [result['rougeL_fmeasure']],
                         'RougeLsum_Precision': [result['rougeLsum_precision']], 'RougeLsum_Recall': [result['rougeLsum_recall']],
                         'RougeLsum_fmeasure': [result['rougeLsum_fmeasure']],
                         'SacreBleu': [result['sacrebleu']], 'Meteor': [result['meteor']]})
## save to xlsx file

filepath = 'top_k401_top_p095.xlsx'
writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
df.to_excel(writer, index=False, sheet_name='Sheet1')
df_result.to_excel(writer, index=False, sheet_name='Result')
writer.save()

In [16]:
result = compute_metrics(reference_question, pred)
print("top_k = 40 top_p=0.95")
result

top_k = 40 top_p=0.95


{'rouge1_precision': 25.8191,
 'rouge1_recall': 26.9641,
 'rouge1_fmeasure': 25.4519,
 'rouge2_precision': 8.7354,
 'rouge2_recall': 9.2826,
 'rouge2_fmeasure': 8.6548,
 'rougeL_precision': 24.4047,
 'rougeL_recall': 25.5485,
 'rougeL_fmeasure': 24.0773,
 'rougeLsum_precision': 24.4055,
 'rougeLsum_recall': 25.579,
 'rougeLsum_fmeasure': 24.1118,
 'sacrebleu': 4.8759,
 'meteor': 0.1388}

In [95]:
result = compute_metrics(reference_question, pred)
print("top_p = 1 temperature=0.95")
result

top_p = 1 temperature=0.95


{'rouge1_precision': 25.3467,
 'rouge1_recall': 26.6861,
 'rouge1_fmeasure': 25.1342,
 'rouge2_precision': 8.1951,
 'rouge2_recall': 8.8998,
 'rouge2_fmeasure': 8.2359,
 'rougeL_precision': 24.0994,
 'rougeL_recall': 25.494,
 'rougeL_fmeasure': 23.9426,
 'rougeLsum_precision': 24.1085,
 'rougeLsum_recall': 25.5208,
 'rougeLsum_fmeasure': 23.9557,
 'sacrebleu': 4.4088,
 'meteor': 0.1363}

In [88]:
result = compute_metrics(reference_question, pred)
print("top_k=100 top_p=0.95")
result

top_k = 100 top_p = 0.95


{'rouge1_precision': 24.3675,
 'rouge1_recall': 25.1895,
 'rouge1_fmeasure': 23.8669,
 'rouge2_precision': 7.7654,
 'rouge2_recall': 8.3323,
 'rouge2_fmeasure': 7.7053,
 'rougeL_precision': 23.1111,
 'rougeL_recall': 23.9016,
 'rougeL_fmeasure': 22.633,
 'rougeLsum_precision': 23.0785,
 'rougeLsum_recall': 23.8937,
 'rougeLsum_fmeasure': 22.6392,
 'sacrebleu': 4.09,
 'meteor': 0.1338}

In [82]:
result = compute_metrics(reference_question, pred)
print("NUM_BEAMS=10")
result

NUM_BEAMS=10


{'rouge1_precision': 33.2037,
 'rouge1_recall': 35.3239,
 'rouge1_fmeasure': 33.1948,
 'rouge2_precision': 14.1174,
 'rouge2_recall': 15.4019,
 'rouge2_fmeasure': 14.2558,
 'rougeL_precision': 31.4554,
 'rougeL_recall': 33.5326,
 'rougeL_fmeasure': 31.4925,
 'rougeLsum_precision': 31.3992,
 'rougeLsum_recall': 33.4731,
 'rougeLsum_fmeasure': 31.4722,
 'sacrebleu': 8.1095,
 'meteor': 0.1794}

In [76]:
result = compute_metrics(reference_question, pred)
print("NUM_BEAMS=7")
result

NUM_BEAMS=7


{'rouge1_precision': 32.9489,
 'rouge1_recall': 35.2039,
 'rouge1_fmeasure': 33.0053,
 'rouge2_precision': 13.9216,
 'rouge2_recall': 15.2639,
 'rouge2_fmeasure': 14.0935,
 'rougeL_precision': 31.2152,
 'rougeL_recall': 33.4568,
 'rougeL_fmeasure': 31.3553,
 'rougeLsum_precision': 31.1402,
 'rougeLsum_recall': 33.4541,
 'rougeLsum_fmeasure': 31.3185,
 'sacrebleu': 8.1074,
 'meteor': 0.1782}

In [69]:
result = compute_metrics(reference_question, pred)
print("NUM_BEAMS=5")
result

NUM_BEAMS=5


{'rouge1_precision': 32.762,
 'rouge1_recall': 35.2326,
 'rouge1_fmeasure': 32.9817,
 'rouge2_precision': 13.9094,
 'rouge2_recall': 15.3718,
 'rouge2_fmeasure': 14.1434,
 'rougeL_precision': 31.159,
 'rougeL_recall': 33.5899,
 'rougeL_fmeasure': 31.4034,
 'rougeLsum_precision': 31.1237,
 'rougeLsum_recall': 33.547,
 'rougeLsum_fmeasure': 31.3612,
 'sacrebleu': 8.0115,
 'meteor': 0.1767}

In [63]:
result = compute_metrics(reference_question, pred)
print("NUM_BEAMS=3")
result

NUM_BEAMS=3


{'rouge1_precision': 32.3829,
 'rouge1_recall': 34.7771,
 'rouge1_fmeasure': 32.5061,
 'rouge2_precision': 13.6391,
 'rouge2_recall': 14.8435,
 'rouge2_fmeasure': 13.7345,
 'rougeL_precision': 30.8244,
 'rougeL_recall': 33.1571,
 'rougeL_fmeasure': 30.9929,
 'rougeLsum_precision': 30.8117,
 'rougeLsum_recall': 33.1441,
 'rougeLsum_fmeasure': 30.9659,
 'sacrebleu': 7.7325,
 'meteor': 0.1743}